In [1]:
import pandas as pd
import numpy as np

# Load Data, Utterance & Label:
df_data = pd.read_csv('../data/Friends_A_whole.tsv', sep = '\t')
df = df_data[['utterance', 'labels']]



def evaluation(validation_dataloader):
    prompt_model.eval()
    labels_list = np.array([])
    pred_list = np.array([])
    for step, inputs in enumerate(validation_dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        preds  = torch.argmax(logits, dim=-1).cpu().tolist()
        labels_list = np.append(labels.cpu().tolist(), labels_list)
        pred_list = np.append(preds, pred_list)
        
        
    
    return f1_score(labels_list, pred_list)




seeds =  [321, 42, 1024, 0, 1, 13, 41, 123, 456, 999]
for SEED in seeds:
    # Train test split:
    from sklearn.model_selection import train_test_split
    Uttr_train, Uttr_test, label_train, label_test = \
        train_test_split(df['utterance'], df['labels'], test_size=0.1, random_state=SEED, stratify=df['labels'])

    Uttr_train, Uttr_valid, label_train, label_valid = \
        train_test_split(Uttr_train, label_train, test_size=0.1, random_state=SEED, stratify=label_train)


    # Construct Samples
    from openprompt.data_utils import InputExample
    dataset = {}
    for split in ['train', 'validation', 'test']:
        dataset[split] = []
        cnt = 0
        if split == 'train':
            for u,l in zip(Uttr_train, label_train):
                input_sample = InputExample(text_a=u, label=int(l),guid=cnt)
                cnt += 1
                dataset[split].append(input_sample)
        elif split == 'validation':
            for u,l in zip(Uttr_valid, label_valid):
                input_sample = InputExample(text_a=u, label=int(l),guid=cnt)
                cnt += 1
                dataset[split].append(input_sample)
        elif split == 'test':
            for u,l in zip(Uttr_test, label_test):
                input_sample = InputExample(text_a=u, label=int(l),guid=cnt)
                cnt += 1
                dataset[split].append(input_sample)


    # Base Model
    from openprompt.plms import load_plm
    plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-cased")



    ## Construct Template and Wrapping
    '''
    #############
    #############
    #############
    '''
    from openprompt.prompts import ManualTemplate
    mytemplate = ManualTemplate(
        text = '{"placeholder":"text_a"} He is a {"mask"} person',
        tokenizer = tokenizer,
    )

    wrapped_example = mytemplate.wrap_one_example(dataset['train'][0])
    wrapped_tokenizer = WrapperClass(max_seq_length=128, tokenizer=tokenizer,truncate_method="head")


    model_inputs = {}
    for split in ['train', 'validation', 'test']:
        model_inputs[split] = []
        for sample in dataset[split]:
            tokenized_example = wrapped_tokenizer.tokenize_one_example(mytemplate.wrap_one_example(sample), teacher_forcing=False)
            model_inputs[split].append(tokenized_example)


    ## Data Loader
    from openprompt import PromptDataLoader

    train_dataloader = PromptDataLoader(dataset=dataset["train"], template=mytemplate, tokenizer=tokenizer,
        tokenizer_wrapper_class=WrapperClass, max_seq_length=128, 
        batch_size=16, shuffle=True, teacher_forcing=False, predict_eos_token=False,
        truncate_method="head")

    validation_dataloader = PromptDataLoader(dataset=dataset["validation"], template=mytemplate, tokenizer=tokenizer,
        tokenizer_wrapper_class=WrapperClass, max_seq_length=128,
        batch_size=16, shuffle=False, teacher_forcing=False, predict_eos_token=False,
        truncate_method="head")

    test_dataloader = PromptDataLoader(dataset=dataset["test"], template=mytemplate, tokenizer=tokenizer,
        tokenizer_wrapper_class=WrapperClass, max_seq_length=128,
        batch_size=16, shuffle=False, teacher_forcing=False, predict_eos_token=False,
        truncate_method="head")


    ## Construct Verbalizer
    '''
    #############
    #############
    #############
    '''
    from openprompt.prompts import ManualVerbalizer
    import torch

    classes = [0,1]
    # https://www.psychometric-assessment.com/the-lexical-hypothesis-and-factor-models/
    myverbalizer = ManualVerbalizer(
                        classes = classes,
                        label_words = {
                            0 : ["disagreeable", "independent",  "dominant", "critical", "demanding", "detached"], 
                            1 : ["agreeable", "compassionate", "trusting", "compliant", "modest", "sensitive"]
                        },
                        tokenizer=tokenizer)

    logits = torch.randn(2, len(tokenizer)) # creating a pseudo output from the plm, and




    from openprompt import PromptForClassification

    use_cuda = True
    prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
    if use_cuda:
        prompt_model =  prompt_model.cuda()



    from transformers import  AdamW, get_linear_schedule_with_warmup
    from sklearn.metrics import f1_score
    from sklearn.metrics import confusion_matrix, classification_report

    loss_func = torch.nn.CrossEntropyLoss()
    no_decay = ['bias', 'LayerNorm.weight']
    # it's always good practice to set no decay to biase and LayerNorm parameters
    optimizer_grouped_parameters = [
        {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]


    ## Training
    '''
    #############
    #############
    #############
    '''
    best_eval = 0
    best_test = 0
    optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)
    for epoch in range(10):
        tot_loss = 0
        for step, inputs in enumerate(train_dataloader):
            if use_cuda:
                inputs = inputs.cuda()
            logits = prompt_model(inputs)
            labels = inputs['label']
            loss = loss_func(logits, labels)
            loss.backward()
            tot_loss += loss.item()
            optimizer.step()
            optimizer.zero_grad()
            if step %100 == 1:
                eval_f1 = evaluation(validation_dataloader)
                if eval_f1 > best_eval:
                    best_eval = eval_f1
                    best_test = evaluation(test_dataloader)
    print('Current SEED:', SEED, 'TEST F1', best_test)




Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (578 > 512). Running this sequence through the model will result in indexing errors
tokenizing: 575it [00:00, 1656.13it/s]
tokenizing: 64it [00:00, 1712.18it/s]
tokenizing: 72it [00:00, 1668.39it/s]
/home/zhiyuan/ENTER/lib/python3.9/site-packages/transformers/optimizatio

Current SEED: 321 TEST F1 0.6190476190476191


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (578 > 512). Running this sequence through the model will result in indexing errors
tokenizing: 575it [00:00, 1636.23it/s]
tokenizing: 64it [00:00, 1639.66it/s]
tokenizing: 72it [00:00, 1533.62it/s]
/home/zhiyuan/ENTER/lib/python3.9/site-packages/transformers/optimizatio

Current SEED: 42 TEST F1 0.6732673267326732


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (578 > 512). Running this sequence through the model will result in indexing errors
tokenizing: 575it [00:00, 1637.49it/s]
tokenizing: 64it [00:00, 1591.80it/s]
tokenizing: 72it [00:00, 1780.17it/s]
/home/zhiyuan/ENTER/lib/python3.9/site-packages/transformers/optimizatio

...

Current SEED: 123 TEST F1 0.6990291262135923


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (578 > 512). Running this sequence through the model will result in indexing errors
tokenizing: 575it [00:00, 1677.81it/s]
tokenizing: 64it [00:00, 1594.28it/s]
tokenizing: 72it [00:00, 1679.35it/s]
/home/zhiyuan/ENTER/lib/python3.9/site-packages/transformers/optimizatio

Current SEED: 456 TEST F1 0.7289719626168225


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (578 > 512). Running this sequence through the model will result in indexing errors
tokenizing: 575it [00:00, 1637.46it/s]
tokenizing: 64it [00:00, 1619.54it/s]
tokenizing: 72it [00:00, 1852.23it/s]
/home/zhiyuan/ENTER/lib/python3.9/site-packages/transformers/optimizatio

Current SEED: 999 TEST F1 0.7102803738317758
